# 📆 Extrator de Consultas - Livro Caixa

Esse Notebook tem como função extrair eventos do Google Calendar e salvá-los em um livro caixa em formato de arquivo Excel.

Nesse projeto é usado o Google Calendar API para extrair do calendário eventos com o nome dos pacientes, esses eventos são tratados e agrupados por recorrência e mês, e são exportados para o livro caixa.


A ideia desse projeto é facilitar a vida do excelente psicólogo [Jean Ghissoni](https://psicologojeanghissoni.negocio.site).


## Primeiros Passos

Para rodar o script basta rodar linha a linha desse notebook, excecutando em ordem cada etapa do código.

## Tabela de conteúdos
1. [Configuração de ambiente](#setup)<br>
    1.1 [Instalando o Cliente Python do Google API](#setup1)<br>
    1.2 [Importando bibliotecas e métodos usandos no notebook](#setup2)<br>
    1.3 [Iniciando variáveis](#setup3)<br>
2. [Autenticando com a API do Google](#auth)<br>
    2.1 [Gerando o arquivo client_secrets.json](#auth1)<br>
    2.2 [Iniciando o Cliente do Google e Autorizando conta Google](#auth2)<br>
3. [Carregando e formatando os dados](#load)<br>
    3.1 [Função de limpeza de Consulta](#load1)<br>
    3.2 [Chamando o serviço que retorna os eventos](#load2)<br>
    3.3 [Agrupando as consultas dos pacientes por mês](#load3)<br>
4. [Gerando o arquivo de Excel](#save)<br>


## <a id="setup"></a>  1. Configuração de ambiente


### <a id="setup1"></a>  1.1 Instalando o Cliente Python do Google API

https://github.com/googleapis/google-api-python-client


In [ ]:
!pip install --upgrade google-api-python-client oauth2client

### <a id="setup2"></a> 1.2 Importando bibliotecas e métodos usandos no notebook

In [ ]:
import pandas as pd
import os
from dateutil import parser
from datetime import datetime
from googleapiclient import sample_tools

### <a id="setup3"></a> 1.3 Iniciando variáveis

In [ ]:
tempo_limite = datetime(2023, 4, 30).isoformat() + 'Z'
tempo_inicio = datetime(2023, 2, 1).isoformat() + 'Z'

lista_mes = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

## <a id="auth"></a> 2. Autenticando com a API do Google


### <a id="auth1"></a> 2.1 Gerando o arquivo client_secrets.json

A biblioteca de cliente de APIs do Google para Python usa o arquivo **client_secrets.json** para armazenar client_id, client_secret e outros parâmetros OAuth 2.0. Dessa forma precisamos dele para conseguirmos nos autenticar com a API do Google.

Consulte [Criando credenciais de autorização](https://developers.google.com/identity/protocols/oauth2/web-server#creatingcred) para saber como obter um arquivo client_secrets.json.

Uma vez gerando o client_secrets.json, é necessário adicionarmos o mesmo uma pasta antes da pasta onde se localiza este notebook.

### <a id="auth2"></a> 2.2 Iniciando o Cliente do Google e Autorizando conta Google

A função *sample_tools.init* ([Documentação](https://googleapis.github.io/google-api-python-client/docs/epy/googleapiclient.sample_tools-module.html)) é usada para inicializar o client do google. Ela usa Oauth2 Flow, com isso, você é redirecionado e precisa se autenticar (pelo menos na primeira vez que executar isso) no site do Google.

Uma vez autenticado, temos na variável **service** o cliente google com seus métodos disponíveis.
 

In [ ]:
print(os.path.abspath(''))

service, flags = sample_tools.init(
        [],
        "calendar",
        "v3",
        __doc__,
        os.path.abspath(''),
        scope="https://www.googleapis.com/auth/calendar.readonly",
    )

## <a id="load"></a> 3. Autenticando com a API do Google


### <a id="load1"></a>3.1 Função de limpeza de Consulta

Recebe um evento e retorna um dicionário com os seguintes campos:
* paciente: Nome do Paciente da consulta
* mes: O mês da consulta
* descricao: A descrição da consulta


In [ ]:
def event_clear(event):
    data = event.get('start', {}).get('dateTime', '')
    month = lista_mes[parser.parse(data).month -1]
    eventDict = { 'paciente': event.get('summary'), 'mes': month, 'descricao': event.get('description')  }
    return eventDict

### <a id="load2"></a>3.2 Chamando o serviço que retorna os eventos

In [ ]:
page_token = None
listaLimpaEventos = []
while True:
  events = service.events().list(calendarId='primary', pageToken=page_token, singleEvents=True, timeMax=tempo_limite, timeMin=tempo_inicio).execute()
  for event in events['items']:
    eventDict = event_clear(event)
    listaLimpaEventos.append(eventDict)
  page_token = events.get('nextPageToken')
  if not page_token:
    break

### <a id="load3"></a>3.3 Agrupando as consultas dos pacientes por mês

In [ ]:
filtred_events = []
for element in listaLimpaEventos:
    if list(filter(lambda event: event['paciente'] == element.get('paciente') and event.get('mes') == element.get('mes'), filtred_events)):
        continue
    element_cnt = 0
    for el in listaLimpaEventos:
        if(element.get('paciente') == el.get('paciente') and element.get('mes') == el.get('mes') ):
            element_cnt += 1
    event_dict = dict(element)
    event_dict['consultas'] = element_cnt
    filtred_events.append(event_dict)

## <a id="save"></a>4 Gerando o arquivo de Excel

https://developers.google.com/calendar/api/v3/reference/events/list

In [ ]:
writer = pd.ExcelWriter('listaPacientes.xlsx', engine = 'openpyxl')

for mes in lista_mes:
    month_filter = list(filter(lambda evt: evt['mes'] == mes, filtred_events))
    df_json = pd.DataFrame.from_dict(month_filter)
    df_json.to_excel(writer, sheet_name = mes,index=False)
writer.close()